In [1]:
using Gync, JLD

In [2]:
sim = Gync.startmcmc(1,100,1,"test.jld", thin=10)
load("test.jld")["chains"]

MCMC Simulation of 100 Iterations x 1 Chain...

Chain 1:   1% [0:03:45 of 0:03:47 remaining]
Chain 1:  10% [0:00:27 of 0:00:30 remaining]
Chain 1:  20% [0:00:13 of 0:00:17 remaining]
Chain 1:  31% [0:00:08 of 0:00:12 remaining]
Chain 1:  40% [0:00:06 of 0:00:10 remaining]
Chain 1:  50% [0:00:04 of 0:00:08 remaining]
Chain 1:  60% [0:00:03 of 0:00:08 remaining]
Chain 1:  70% [0:00:02 of 0:00:07 remaining]
Chain 1:  80% [0:00:01 of 0:00:06 remaining]
Chain 1:  90% [0:00:01 of 0:00:06 remaining]
Chain 1: 100% [0:00:00 of 0:00:06 remaining]



10x115x1 Array{Float64,3}:
[:, :, 1] =
 7057.47  6947.57  185.002  2.17944  …  0.00087798   3.78975e-5  2.94864e-5
 7057.47  6947.57  185.002  2.17944     0.00087798   3.78975e-5  2.94864e-5
 7058.32  6813.99  170.127  2.32029     0.000951024  3.52344e-5  2.52823e-5
 6542.29  7338.12  170.419  1.97948     0.000979322  4.55581e-5  2.56259e-5
 6542.29  7338.12  170.419  1.97948     0.000979322  4.55581e-5  2.56259e-5
 5617.35  7038.63  162.422  1.87444  …  0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5

In [3]:
Gync.continuemcmc("test.jld", 100, thin=10)
load("test.jld")["chains"]

MCMC Simulation of 100 Iterations x 1 Chain...

Chain 1:   1% [0:00:02 of 0:00:02 remaining]
Chain 1:  10% [0:00:02 of 0:00:02 remaining]
Chain 1:  20% [0:00:02 of 0:00:02 remaining]
Chain 1:  31% [0:00:02 of 0:00:02 remaining]
Chain 1:  40% [0:00:02 of 0:00:03 remaining]
Chain 1:  50% [0:00:01 of 0:00:03 remaining]
Chain 1:  60% [0:00:01 of 0:00:03 remaining]
Chain 1:  70% [0:00:01 of 0:00:03 remaining]
Chain 1:  80% [0:00:01 of 0:00:03 remaining]
Chain 1:  90% [0:00:00 of 0:00:03 remaining]
Chain 1: 100% [0:00:00 of 0:00:03 remaining]



20x115x1 Array{Float64,3}:
[:, :, 1] =
 7057.47  6947.57  185.002  2.17944  …  0.00087798   3.78975e-5  2.94864e-5
 7057.47  6947.57  185.002  2.17944     0.00087798   3.78975e-5  2.94864e-5
 7058.32  6813.99  170.127  2.32029     0.000951024  3.52344e-5  2.52823e-5
 6542.29  7338.12  170.419  1.97948     0.000979322  4.55581e-5  2.56259e-5
 6542.29  7338.12  170.419  1.97948     0.000979322  4.55581e-5  2.56259e-5
 5617.35  7038.63  162.422  1.87444  …  0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444  …  0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.00112207   3.88283e-5  2.05411e-5
 5617.35  7038.63  162.422  1.87444     0.0011220

In [ ]:
using JLD
include("Gync.jl");
parms, y0 = loadparms()
data = loadpfizer();

SIGMA_RHO = 0.1
SIGMA_Y0  = 1
SIGMA_PARMS = 20
SIGMA_PROPOSAL = 0.1
adapt = :all
stepsize = 100
sampler = :amm
persons = 1
samplingnodes = [:sparms, :y0]
chains = 5


jldname = "$sampler $adapt ugm-flat $(length(persons))pX$(chains)c $(SIGMA_RHO)r $(SIGMA_Y0)sy $(SIGMA_PARMS)sp $(SIGMA_PROPOSAL)p.jld"

d = eye(length(SAMPLEPARMS)+length(y0))
proposalvariance = d * log(1+(SIGMA_PROPOSAL^2));

samplers = Dict(
    :amm=>[AMM(samplingnodes, proposalvariance, adapt=adapt)], 
    :amwg=>[AMWG(samplingnodes,diag(proposalvariance); target=0.44)])

mcs = []
for person in persons
    m, inp, ini = gyncmodel(data[person], parms, y0)
    setsamplers!(m, samplers[sampler])
    
    push!(mcs, mcmc(m, inp, [ini[1] for i=1:chains], stepsize, burnin=0, verbose = false, thin = 1, chains=chains))
end

#samplelength(mc) = max(floor(Int, size(mc, 1)/5), 2_000)

println(jldname)
while size(mcs[1],1) < 1_000_000
    @time for i in 1:length(persons)
        mcs[persons[i]] = mcmc(mcs[persons[i]], stepsize, verbose=false)
    end
    @time save(jldname, "mc", [mc.value for mc in mcs])    
    println(size(mcs[1]))
end